In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install transformers==3.5.1

     |████████████████████████████████| 1.3MB 13.0MB/s 
     |████████████████████████████████| 1.1MB 48.8MB/s 
     |████████████████████████████████| 2.9MB 48.8MB/s 
     |████████████████████████████████| 890kB 53.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=eb91cce33d1394b09e4063fd57e9369d0ddb5421d9d396ca3977e957b1502650
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 24.1MB/s 
     |████████████████████████████████| 1.8MB 52.5MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 317kB 49.8MB/s 
     |████████████████████████████████| 7.4MB 39.0MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 1.4MB 50.5MB/s 
     |████████████████████████████████| 163kB 62.6MB/s 
     |████████████████████████████████| 133kB 61.0MB/s 
     |████████████████████████████████| 102kB 15.6MB/s 
     |████████████████████████████████| 102kB 15.8MB/s 
     |████████████████████████████████| 112kB 58.6MB/s 
     |████████████████████████████████| 4.5MB 47.5MB/s 
     |████████████████████████████████| 71kB 12.5MB/s 
     |████████████████████████████████| 122kB 55.1MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=cd37aade1fc463c8267719cf37351daeb1aa18a2e029eb19546e4b8aff2c274c
  Stored in

In [ ]:
!pip install tokenizers==0.9.4

     |████████████████████████████████| 2.9MB 13.3MB/s 
  Found existing installation: tokenizers 0.9.3
    Uninstalling tokenizers-0.9.3:
      Successfully uninstalled tokenizers-0.9.3


In [ ]:
!pip install datasets

     |████████████████████████████████| 163kB 17.2MB/s 
     |████████████████████████████████| 17.7MB 216kB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
     |████████████████████████████████| 245kB 60.6MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
  Found existing installation: tqdm 4.54.0
    Uninstalling tqdm-4.54.0:
      Successfully uninstalled tqdm-4.54.0


In [ ]:
from datasets import load_dataset
dataset = load_dataset("wikisql")
print(dataset['train'][0])
# train = []
# with open('/content/train',"w") as x:
#   for d in dataset["train"]:
#     line = d["question"]+"[SEP]"+d["sql"]["human_readable"]+'\n'
#     x.write(line)
# validation = []
# with open('/content/val', "w") as x:
#   for d in dataset["validation"]:
#     line = d["question"]+"[SEP]"+d["sql"]["human_readable"]+'\n'
#   x.write(line)
# test = []
# with open('/content/test', "w") as x:
#   for d in dataset["test"]:
#     line = d["question"]+"[SEP]"+d["sql"]["human_readable"]+'\n'
#   x.write(line)



Using custom data configuration default


Dataset wiki_sql downloaded and prepared to /root/.cache/huggingface/datasets/wiki_sql/default/0.1.0/2e98053891fd8f9b2c4348bba609ce40cc0a4d7f621191cebcd7cb558b5f8a70. Subsequent calls will reuse this data.
{'phase': 1, 'question': 'Tell me what the notes are for South Australia ', 'sql': {'agg': 0, 'conds': {'column_index': [3], 'condition': ['SOUTH AUSTRALIA'], 'operator_index': [0]}, 'human_readable': 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA', 'sel': 5}, 'table': {'caption': '', 'header': ['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes'], 'id': '1-1000181-1', 'name': 'table_1000181_1', 'page_id': '', 'page_title': '', 'rows': [['Australian Capital Territory', 'blue/white', 'Yaa·nna', 'ACT · CELEBRATION OF A CENTURY 2013', 'YIL·00A', 'Slogan screenprinted on plate'], ['New South Wales', 'black/yellow', 'aa·nn·aa', 'NEW SOUTH WALES', 'BX·99·HI', 'No slogan on current series'], ['New South Wales', 'black/white', '

In [ ]:
train = []
for d in dataset["train"]:
  temp = []
  temp.append(d['question'])
  temp.append(d["sql"]["human_readable"])
  train.append(temp)
  # break
# print(train)
validation = []
for d in dataset["validation"]:
  temp = []
  temp.append(d['question'])
  temp.append(d["sql"]["human_readable"])
  validation.append(temp)

In [ ]:
import logging

import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
# train_data = [
#     ["one", "1"],
#     ["two", "2"],
# ]

train_df = pd.DataFrame(train, columns=["input_text", "target_text"])

# eval_data = [
#     ["three", "3"],
#     ["four", "4"],
# ]
print(validation[0])

eval_df = pd.DataFrame(validation, columns=["input_text", "target_text"])

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 50,
    "train_batch_size": 32,
    "num_train_epochs": 30,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "evaluate_generated_text": True,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "max_length": 25,
    "manual_seed": 4,
    # "weight_decay":0.01,
    # "learning_rate":8e-5,
    # "warmup_steps":1000,
    "output_dir": '/content/drive/MyDrive/685_Project/final_outputs_distilbert',
}
encoder_type = "distilbert"

model = Seq2SeqModel(
    encoder_type,
    "distilbert-base-cased",
    "bert-base-cased",
    args=model_args,
    use_cuda=True,
)


INFO:filelock:Lock 140454822108856 acquired on /root/.cache/huggingface/transformers/ebe1ea24d11aa664488b8de5b21e33989008ca78f207d4e30ec6350b693f073f.302bfd1b5e031cc1b17796e0b6e5b242ba2045d31d00f97589e12b458ebff27a.lock


['What position does the player who played for butler cc (ks) play?', 'SELECT Position FROM table WHERE School/Club Team = Butler CC (KS)']


INFO:filelock:Lock 140454822108856 released on /root/.cache/huggingface/transformers/ebe1ea24d11aa664488b8de5b21e33989008ca78f207d4e30ec6350b693f073f.302bfd1b5e031cc1b17796e0b6e5b242ba2045d31d00f97589e12b458ebff27a.lock
INFO:filelock:Lock 140457620812240 acquired on /root/.cache/huggingface/transformers/9c9f39769dba4c5fe379b4bc82973eb01297bd607954621434eb9f1bc85a23a0.06b428c87335c1bb22eae46fdab31c8286efa0aa09e898a7ac42ddf5c3f5dc19.lock


INFO:filelock:Lock 140457620812240 released on /root/.cache/huggingface/transformers/9c9f39769dba4c5fe379b4bc82973eb01297bd607954621434eb9f1bc85a23a0.06b428c87335c1bb22eae46fdab31c8286efa0aa09e898a7ac42ddf5c3f5dc19.lock


INFO:filelock:Lock 140455093696720 acquired on /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655.lock


INFO:filelock:Lock 140455093696720 released on /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655.lock
INFO:filelock:Lock 140454821997760 acquired on /root/.cache/huggingface/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda.lock


INFO:filelock:Lock 140454821997760 released on /root/.cache/huggingface/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda.lock


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.c

INFO:filelock:Lock 140454818057968 released on /root/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791.lock
INFO:filelock:Lock 140455094004592 acquired on /root/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6.lock


INFO:filelock:Lock 140455094004592 released on /root/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6.lock


In [ ]:


model.train_model(train_df)



INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.seq2seq.seq2seq_utils: Saving features into cached file cache_dir/distilbert-base-cased-bert-base-cased_cached_5056355


INFO:simpletransformers.seq2seq.seq2seq_model: Training started


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into /content/drive/MyDrive/685_Project/final_outputs_distilbert/checkpoint-2000
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


...

INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into /content/drive/MyDrive/685_Project/final_outputs_distilbert/checkpoint-48000


INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into /content/drive/MyDrive/685_Project/final_outputs_distilbert/checkpoint-50000


INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into /content/drive/MyDrive/685_Project/final_outputs_distilbert/checkpoint-52000
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into /content/drive/MyDrive/685_Project/final_outputs_distilbert


INFO:simpletransformers.seq2seq.seq2seq_model: Training of distilbert-base-cased-bert-base-cased model complete. Saved to /content/drive/MyDrive/685_Project/final_outputs_distilbert.


(52860, 0.17489604988397134)

In [ ]:
test = []
for d in dataset["test"]:
  temp = []
  temp.append(d['question'])
  temp.append(d["sql"]["human_readable"])
  test.append(temp)
print(test[0])
print(test[1])
print(test[40])

["What is terrence ross' nationality", 'SELECT Nationality FROM table WHERE Player = Terrence Ross']
['What clu was in toronto 1995-96', 'SELECT School/Club Team FROM table WHERE Years in Toronto = 1995-96']
['Which Frequency is used for WEGP calls?', 'SELECT Frequency FROM table WHERE Calls = WEGP']


In [ ]:
results = model.eval_model(eval_df)

print(model.predict(["What clu was in toronto 1995-96"]))


# model1 = Seq2SeqModel(
#     encoder_type,
#     encoder_decoder_name="outputs",
#     args=model_args,
#     use_cuda=True,
# )
# print(model1.predict(["What clu was in toronto 1995-96"])

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.seq2seq.seq2seq_utils: Saving features into cached file cache_dir/distilbert-base-cased-bert-base-cased_cached_508421


INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.5111238998555907}


['SELECT CluBo 1995 - 96 FROM table WHERE College = toronto 1995']


In [ ]:
print(model.predict(["Which Frequency is used for WEGP calls?"]))

['SELECT Frequency FROM table WHERE Callsign = wpc.']


In [ ]:
!zip /content/outputs.zip /content/outputs

  adding: content/outputs/ (stored 0%)
